In [28]:
import pandas as pd
import folium
from pymongo import MongoClient, GEO2D

In [29]:
client = MongoClient('mongodb://localhost:27017/')
db = client.offices
datos_mongo = db.offices


puntuaciones= {
    'games_video' : 100,
    'software' : 50,
    'BigCompany' : 25,
    'Startup' : 40,
    'USA': -80
    }

In [30]:
# Obtener datos de mongo con el indice 2d geo.
def get_geo_data(datos):
    cursor_geo= datos.find()
    df = pd.DataFrame(cursor_geo)
    df = df.drop('_id', axis=1)
    return df

In [31]:
def sum_points(reg):
    points = 0
    points+= puntuaciones[reg.category_code]
    points+= puntuaciones[reg.type]
    if reg.country_code== 'USA': points+= puntuaciones['USA']
    return points

def get_offices_near(reg):
    points= list()
    lat = float(reg.latitude)
    long = float(reg.longitude)
    df_location = pd.DataFrame(datos_mongo.find({
                                "2Dgeo": {
                                 "$near": {
                                   "$geometry": {
                                      "type": "Point" ,
                                      "coordinates": [ long, lat ]
                                   },
                                   "$maxDistance": 3000, # In meters
                                 }
                               }
                            }))
    points.extend(df_location.apply(sum_points, axis=1))
    return sum(points)

In [32]:
def calculate_points(df):
    df['points'] = df.apply(get_offices_near, axis=1)
    return df

In [33]:
#Main
df_geo_mongo = get_geo_data(datos_mongo)

In [34]:
df_points = calculate_points(df_geo_mongo)
df_points= df_points.sort_values('points', ascending=False)

#TODO pintar el punto en el mapa.

In [35]:
df_points.head()

,2Dgeo,category_code,city,country_code,founded_year,latitude,longitude,name,number_of_employees,type,points
71,"[-0.1247662, 51.5111369]",software,London,GBR,2006,51.511137,-0.124766,Zmags,95,BigCompany,1450
432,"[-0.1158832, 51.5123833]",software,London,GBR,2007,51.512383,-0.115883,Emoshape,10,BigCompany,1450
640,"[-0.1153003, 51.5220763]",games_video,London,GBR,2007,51.522076,-0.115300,Keen Software House,5,BigCompany,1310
386,"[-0.11796, 51.5178123]",software,London,GBR,2007,51.517812,-0.117960,FreshNetworks,12,BigCompany,1310
608,"[-0.1262362, 51.5001524]",games_video,London,GBR,2008,51.500152,-0.126236,CareerPlayer,5,Startup,1310


In [38]:
latitud= float(df_points[0:1].latitude)
longitud= float(df_points[0:1].longitude)

-0.1247662

In [39]:
m = folium.Map(location=[latitud, longitud])
m.save('index.html')

In [40]:
m